In [ ]:
!pip install -q -U keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 5.8 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import seaborn as sns
#import datetime as dt
import matplotlib.pyplot as plt
import keras_tuner as kt
import io, os, math, gc
from tensorflow import keras
from keras.optimizers import Adam
from keras.models import Sequential, save_model, load_model
from keras.layers import Dense, Activation, Input
from keras.callbacks import EarlyStopping, History
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import classification_report, ConfusionMatrixDisplay

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#How to Speed Up Reading Parquet Files Using PyArrow in Python
#import pyarrow.parquet as pq
#data = pq.read_table('Sample.parquet')
#df = data.to_pandas()

#read parquet file in a pandas dataframe
# path = '/content/drive/MyDrive/dataset_zip/CSE-CIC-2018/cleaning_file/CICIDS2018_preprocessing_binary.parquet'
path= '/content/drive/MyDrive/dataset_zipped-20230608T100425Z-005/dataset_zipped/CSE-CIC-2018/cleaning_file/CICIDS2018_preprocessing_binary.parquet'
df = pd.read_parquet(path)
print (df.shape)
#print(df.columns)

(7384563, 67)


In [ ]:
#read parquet file in a pandas dataframe
path = '/content/drive/MyDrive/dataset/df_total_preprocessing_binarized.parquet'
df = pd.read_parquet(path)
#print (df.shape)

In [ ]:
'''
#analyzing dataset

#info
print('='*50)
print('DATAFRAME analysis... ')
print (df.describe())
print("Read {} rows.".format(len(df[' Label'])))
print('='*50)
print('Column in a dataframe:', len(df.columns))
print('='*50)
print('The number of features are:', len(df.columns)-1)
print('='*50)
labels = df.groupby([' Label'])[' Label'].count()
print('The number of output labels is {}:', len(labels))
print('='*50)
print('Label distribution:', labels)

#figure
plt.figure(figsize=(20,10))
class_distribution = labels.value_counts()
class_distribution.plot(kind='bar')
plt.xlabel('Label')
plt.ylabel('Data points per Label')
plt.title('yi distribution in train data')
plt.grid()
plt.show()
'''

'\n#analyzing dataset\n\n#info\nprint(\'=\'*50)\nprint(\'DATAFRAME analysis... \')\nprint (df.describe())\nprint("Read {} rows.".format(len(df[\' Label\'])))\nprint(\'=\'*50)\nprint(\'Column in a dataframe:\', len(df.columns))\nprint(\'=\'*50)\nprint(\'The number of features are:\', len(df.columns)-1)\nprint(\'=\'*50)\nlabels = df.groupby([\' Label\'])[\' Label\'].count()\nprint(\'The number of output labels is {}:\', len(labels))\nprint(\'=\'*50)\nprint(\'Label distribution:\', labels)\n\n#figure\nplt.figure(figsize=(20,10))\nclass_distribution = labels.value_counts()\nclass_distribution.plot(kind=\'bar\')\nplt.xlabel(\'Label\')\nplt.ylabel(\'Data points per Label\')\nplt.title(\'yi distribution in train data\')\nplt.grid()\nplt.show()\n'

In [ ]:
#Divide features and label
x_columns = df.columns.drop(' Label')
x = df[x_columns]
#y = pd.get_dummies(df[' Label']) # Classification
y = df[' Label']

KeyError: ignored

In [ ]:
#apply PCA reduction
print ("X shape before PCA application --> {}".format(x.shape))

pca = PCA(n_components="mle") #Minka’s MLE is used to guess the dimension
x = pca.fit_transform(x)

print ("X shape after PCA application --> {}".format(x.shape))

NameError: ignored

In [ ]:
#Split dataframe in train/test
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)
print ("X_train --> {} y_train --> {}".format(x_train.shape, y_train.shape))
print ("X_test --> {} y_test --> {}".format(x_test.shape, y_test.shape))
del df
#print (y_test.head(10))

X_train --> (7343852, 57) y_train --> (7343852,)
X_test --> (3147366, 57) y_test --> (3147366,)


In [ ]:
def model_builder(hp):
  model = keras.Sequential()

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-128
  hp_units = hp.Int('units', min_value=32, max_value=64, step=16)
  model.add(Dense(x.shape[1], input_dim=x.shape[1], kernel_initializer='normal', activation='relu'))
  model.add(Dense(units=hp_units, input_dim=x.shape[1], kernel_initializer='normal', activation='relu'))

  model.add(Dense(1,activation='sigmoid'))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=['accuracy'])
  model.summary()
  return model

In [ ]:
#TUNER instance and stop_early definition,
max_epochs = 10
factor = 3
print ("Estimated_trials = ", 1 + (max_epochs * (math.log(max_epochs, factor) ** 2)))

tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs= max_epochs,
                     factor=factor,
                     directory='/content/drive/MyDrive/dataset/',
                     project_name='binary_model_tuning')

tuner.search_space_summary()
stop_early = EarlyStopping(monitor='val_loss', patience=2, verbose=1, mode='min', baseline=0.2)

Estimated_trials =  44.92810535176964
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                1856      
                                                                 
 dense_1 (Dense)             (None, 32)                1056      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2,945
Trainable params: 2,945
Non-trainable params: 0
_________________________________________________________________
Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 64, 'step': 16, 'sampling': 'linear'}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [ ]:
tuner.search(x_train, y_train, epochs=max_epochs, callbacks=[stop_early], validation_data=(x_test, y_test), batch_size=10000)

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters()[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

tuner.results_summary()

Trial 9 Complete [00h 00m 13s]
val_accuracy: 0.9985130429267883

Best val_accuracy So Far: 0.9990830421447754
Total elapsed time: 00h 03m 05s

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 48 and the optimal learning rate for the optimizer
is 0.01.

Results summary
Results in /content/drive/MyDrive/dataset/binary_model_tuning
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 0006 summary
Hyperparameters:
units: 48
learning_rate: 0.01
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 2
tuner/round: 0
Score: 0.9990830421447754

Trial 0001 summary
Hyperparameters:
units: 32
learning_rate: 0.01
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 2
tuner/round: 0
Score: 0.9990207552909851

Trial 0000 summary
Hyperparameters:
units: 64
learning_rate: 0.01
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 2
tuner/round: 0
Score: 0.9990083575248718

Trial 0002 summary
Hyperparameters:
units: 64
l

In [ ]:
#train a model
if os.path.isdir('/content/drive/MyDrive/dataset/binary_model_saved'):
  model = load_model('/content/drive/MyDrive/dataset/binary_model_saved')
else:
  model = tuner.hypermodel.build(best_hps)
  monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto')
  model.fit(x_train,y_train,validation_data=(x_test,y_test),callbacks=[monitor],verbose=2,epochs=10, batch_size=1000)
  save_model(
    model,
    '/content/drive/MyDrive/dataset/binary_model_saved',
    overwrite=True,
    include_optimizer=True
  )
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 48)                2784      
                                                                 
 dense_7 (Dense)             (None, 48)                2352      
                                                                 
 dense_8 (Dense)             (None, 1)                 49        
                                                                 
Total params: 5,185
Trainable params: 5,185
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
7344/7344 - 29s - loss: 0.0031 - accuracy: 0.9988 - val_loss: 0.0021 - val_accuracy: 0.9991 - 29s/epoch - 4ms/step
Epoch 2/10
7344/7344 - 26s - loss: 0.0020 - accuracy: 0.9990 - val_loss: 0.0019 - val_accuracy: 0.9990 - 26s/epoch - 4ms/step
Epoch 3/10
7344/7344 - 23s - loss: 0.0019 - accuracy: 0.9990 - v

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 48)                2784      
                                                                 
 dense_7 (Dense)             (None, 48)                2352      
                                                                 
 dense_8 (Dense)             (None, 1)                 49        
                                                                 
Total params: 5,185
Trainable params: 5,185
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
#confusion matrix definition
def confusion_plot(y_test, y_test_pred):

    C = confusion_matrix(y_test, y_test_pred)
    cm_df = pd.DataFrame(C)
    labels = pd.unique(df[' Label'].values)
    plt.figure(figsize=(20,15))
    sns.set(font_scale=1.4)
    sns.heatmap(cm_df, annot=True, annot_kws={"size":12}, fmt='g', xticklabels=labels, yticklabels=labels)
    plt.ylabel('Actual Class')
    plt.xlabel('Predicted Class')

    plt.show()

In [ ]:
#test a model
escore = model.evaluate(x_test, y_test, batch_size=32)
y_pred = model.predict(x_test)
#y_train_pred = model.predict(x_train)


98356/98356 [==============================] - 102s 1ms/step


In [ ]:
# Confusion Matrix
y_pred = best_model.predict(x_test)
y_pred_classes = (y_pred > 0.5).astype(int)
cm = confusion_matrix(y_test, y_pred_classes)

# Plot confusion matrix
plt.figure(figsize=(6, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False, square=True)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

#print a results

#y_test = y_test.to_numpy().reshape(-1,1)
y_pred = y_pred.round().astype(int)

target_names = ['Begnign', 'Ddos attack']
print(classification_report(y_test, y_pred, target_names=target_names))

disp = metrics.ConfusionMatrixDisplay.from_predictions(y_test, y_pred)
disp.figure_.suptitle("Confusion Matrix")
print(f"Confusion matrix:\n{disp.confusion_matrix}")

plt.show()

vscore = accuracy_score(y_test,y_pred)
rscore = recall_score(y_test, y_pred, average='weighted')
ascore = precision_score(y_test, y_pred, average='weighted')
f1score= f1_score(y_test, y_pred, average='weighted')

print('Completed')
print('='*50)
print("Accuracy score: {}".format(vscore))
print('='*50)
print("Evaluation score: {}".format(escore))
print('='*50)
print("Recall score: {}".format(rscore))
print('='*50)
print("Precision score: {}".format(ascore))
print('='*50)
print("F1 score: {}".format(f1score))
print('='*50)